In [8]:
import pandas as pd
import numpy as np
from scipy import stats

import plotly.express as px
import plotly

import glob
import datetime
from pprint import pprint

pd.set_option("display.max_colwidth", 3000)

In [10]:
df = pd.concat([pd.read_csv(csv, sep=";") for csv in glob.glob("*.csv")])

df['datetime'] = pd.to_datetime(df.date)

# df['year'] = df['datetime'].dt.year
# df['month'] = df['datetime'].dt.month
# df['day'] = df['datetime'].dt.day
# df['hour'] = df['datetime'].dt.hour
# df['minute'] = df['datetime'].dt.minute
# df['second'] = df['datetime'].dt.second
# df['utc_offset'] = df['datetime'].apply(lambda x: x.utcoffset().total_seconds())

# seasons_map = {
#     1 : 'winter',
#     2 : 'winter',
#     3 : 'spring',
#     4 : 'spring',
#     5 : 'spring',
#     6 : 'summer',
#     7 : 'summer',
#     8 : 'summer',
#     9 : 'fall',
#     10 : 'fall',
#     11 : 'fall',
#     12 : 'winter',
# }
# df['season'] = df.month.replace(seasons_map)

df = df.drop(["date"], axis=1)

df = df.rename(columns={"Categoría": "category"})

df = df.drop(['title'], axis=1)

df.head()

,amount,category,datetime
0,-34.13,compras,2017-12-27 20:30:12+00:00
1,-8.37,compras,2018-03-11 00:00:00+00:00
2,-27.77,compras,2017-11-25 05:21:29+00:00
3,2624.84,transferencias,2017-10-30 17:33:03+00:00
4,-31.15,transferencias,2017-12-31 09:07:15+00:00


## Criterios
- La distribución temporal de la muestra debe asemejarse a la de la población.
- Estadísticos descriptivos de `amount` en la muestra se asemejan a los de la población.
- Los clusterings aplicados a la muestra y a la población se asemejan.
- La distribución de tokens de `title` de la muestra debe ser similar a la de la población.
- [OPCIONAL] La distribución de `category` de la muestra debe ser similar a la de la población.

In [11]:
DROP_CATEGORY = False

if DROP_CATEGORY:
    df = df.drop('category', axis=1)

In [12]:
from evaluation.core import Evaluation
from evaluation.evaluations import DistributionComparisson
from evaluation.sampling import RandomSampling

sampling_strategies = [
    RandomSampling(),
]

evaluation_strategy = DistributionComparisson()

evaluation = Evaluation(df = df, sampling_strategies = sampling_strategies, evaluation_strategy = evaluation_strategy)

In [13]:
e = evaluation.run(random_state=42)

In [14]:
e

,amount,category,datetime,SampleSize,SamplingStrategy
0,0.552700,0.614120,0.723307,1,RandomSampling
1,0.167601,0.506121,0.748405,51,RandomSampling
2,0.089043,0.863431,0.275315,101,RandomSampling
3,0.059290,0.705740,0.860834,151,RandomSampling
4,0.070112,0.936526,0.527470,201,RandomSampling
...,...,...,...,...,...
395,0.000455,0.444863,0.210444,19751,RandomSampling
396,0.000441,0.511621,0.720543,19801,RandomSampling
397,0.000385,0.886172,0.134556,19851,RandomSampling
398,0.000400,0.567459,0.611883,19901,RandomSampling


In [15]:
evaluation.plot()